In [ ]:
import numpy as np
np.random.seed(100)
import matplotlib.pyplot as plt

# --------------------------
# 1. Simulation Parameters
# --------------------------
Nx, Ny = 30, 30          # Grid resolution
# Nx, Ny = 5, 5          # Grid resolution
Lx, Ly = 2.0, 2.0        # Physical domain size
dx = Lx / (Nx - 1)
dy = Ly / (Ny - 1)
nu = 0.1                 # Relaxation speed
dt = 0.01                # Time step (ensure stability)
nsteps = 10000       # Number of time steps

# Create grid for plotting
x_vals = np.linspace(0, Lx, Nx)
y_vals = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x_vals, y_vals, indexing='ij')

# --------------------------
# 2. Initialize Director Field
# --------------------------
# TODO: Initialize the director field with random orientations and normalize them
def initial_f():
    grid = np.zeros([Nx, Ny, 2])
    angles = np.random.uniform(0,2*np.pi, [Nx, Ny])
    grid[:,:,0] = np.cos(angles)
    grid[:,:,1] = np.sin(angles)
    return grid


def plot_grid(grid, fig=None, ax=None):
    if fig is None:
        _, ax = plt.subplots(1,1)
    ax.quiver(X, Y, grid[:,:,0].flatten(), grid[:,:,1].flatten())
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    if fig is None: 
        plt.show()


# plot_grid(initial_f())


# --------------------------
# 3. Laplacian Function
# --------------------------
def laplace_2d(f):
    """
    Computes the 2D Laplacian of field f using a 5-point stencil.
    TODO: Implement the Laplacian using np.roll for periodic boundaries.
    """
    # TODO: Shift f in the positive and negative x-directions
    
    # TODO: Shift f in the positive and negative y-directions

    # TODO: Compute the Laplacian using the 5-point stencil
    
    
    lap_f = np.zeros_like(f)
    
    lap_f[1:-1, 1:-1] = 1/dx**2 *(-2*f[1:-1,1:-1] + f[1:-1,0:-2]+ f[1:-1,2:]) + 1/dy**2 *(-2*f[1:-1,1:-1] + f[0:-2,1:-1]+ f[2:,1:-1])
   
    # TODO: Update periodic boundary conditions if needed
    return lap_f


def grad_2d(f):
    
    grad_f = np.zeros_like(f)
    
    grad_f[1:-1, 1:-1] = 0.5 * (f[2:, 1:-1] - f[0:-2, 1:-1]) +  0.5 * (f[1:-1, 2:] - f[1:-1, 0:-2])
    
    return grad_f
    


# --------------------------
# 4. Boundary Conditions
# --------------------------
def set_boundary_conditions(f, bc_type):
    """
    Sets the boundary conditions for field f based on bc_type.
    TODO: Implement both 'periodic' and 'open' (Neumann) boundary conditions.
    """
    f[0,0] = 0
    f[0,-1] = 0
    f[-1,0] = 0
    f[-1,-1] = 0
    if bc_type == 'periodic':
        # TODO: For periodic boundaries, set the edges using np.roll or direct assignment.
        f[0,1:-1] = f[-2,1:-1]
        f[-1,1:-1] = f[1,1:-1]
        f[1:-1,0] = f[1:-1,-2]
        f[1:-1,-1] = f[1:-1,0]
        
        pass
    elif bc_type == 'open':
        # TODO: For open (Neumann) boundaries....think about it
        f[0,1:-1] = f[1, 1:-1]
        f[-1,1:-1] = f[-2, 1:-1]
        f[1:-1, 0] = f[1:-1, 1]
        f[1:-1, -1] = f[1:-1, -2]
        
    return f

# --------------------------
# 5. Minimization Function
# --------------------------
def minimize_frank_energy(n, nsteps, dt, nu, bc_type):
    """
    Evolves the director field n according to:
        ∂n/∂t = nu (∇²n - n (n · ∇²n))
    while maintaining |n| = 1.
    
    TODO: Complete this function by performing the following steps:
          1. Compute the Laplacian for each component of n.
          2. Optionally apply boundary conditions using set_boundary_conditions.
          3. Project the Laplacian update so that the update is perpendicular to n.
          4. Update the field using an explicit Euler time-stepping scheme.
          5. Renormalize the director field at each time step.
    """
    # energies = np.zeros(nsteps)
    set_boundary_conditions(n, bc_type)
    energies = np.zeros(nsteps)
    for step in range(nsteps):
        # Extract components of the director field
        
        lap_n = laplace_2d(n)
        # update = lap_n
        # x = np.sum(n * lap_n, axis=-1, keepdims=True)
        # update -=  n* x
        # n += nu*dt*update
        n += nu * dt * (lap_n - n* np.sum(n * lap_n, axis=-1, keepdims=True))
        
        n = n / np.linalg.norm(n, axis=-1, keepdims=True)
        
        grad_n = grad_2d(n)
        E = nu / (2*(Ny-1)*(Nx-1)) * np.sum(grad_n * grad_n)
        energies[step] = E
        print(E)
        set_boundary_conditions(n, bc_type)
        
        # plot_grid(n)
        
        
                

    return n

# # --------------------------
# # 6. Simulation and Visualization
# # --------------------------
# # TODO: Plot the initial director field using a quiver plot
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.quiver(...)
# plt.title("Initial Director Field")
# plt.axis('equal')
# plt.show()

# # TODO: Perform minimization using your minimize_frank_energy function.
# # Change 'bc_type' to 'periodic' or 'open' as needed.
n = initial_f()

n_final = minimize_frank_energy(n.copy(), nsteps, dt, nu, bc_type='periodic')

# # TODO: Plot the final director field using a quiver plot
fig, axs = plt.subplots(1, 2, figsize=(20,10))
plot_grid(n, fig, axs[0])
plot_grid(n_final, fig, axs[1])
plt.show()



n_final = minimize_frank_energy(n.copy(), nsteps, dt, nu, bc_type='open')

# # TODO: Plot the final director field using a quiver plot
fig, axs = plt.subplots(1, 2, figsize=(20,10))
plot_grid(n, fig, axs[0])
plot_grid(n_final, fig, axs[1])
plt.show()
# plt.quiver(...)
# plt.title(...)
# plt.axis(...)

# plt.tight_layout()
# plt.show()
